# **Modelling and Evaluation**

## Objectives

* Fulfil Business Requirement 2: Accurately predict if a cherry leaf is healthy or contains powdery mildew
* Answer hypothesis #1: Mildew infected leaves will have clear signs of white mildew that can differentiate them from a healthy leaf.

## Inputs

* inputs/datasets/raw/cherry-leaves/train
* inputs/datasets/raw/cherry-leaves/validation
* inputs/datasets/raw/cherry-leaves/test
* image shape .pkl files

## Outputs

* Image augmentation of training dataset
* CNN (Convolutional Neural Network) model tuning using kears-tuner
* Tuned CNN model creation and training.
* Trained model saved and exported to outputs folder
* Learning curve plots for model performance.
* Model evaluation with confusion matrix plot
* Test prediction on random single image file.

## Additional Comments | Insights | Conclusions

* 



---

# Setup

## Library Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras_tuner as kt
import joblib
import shutil
import random
from matplotlib.image import imread
from IPython.display import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

## Set working directory

Change the working directory from its current folder to its parent folder

In [2]:
current_dir = os.getcwd()
current_dir

'/workspace/cherry-mildew-detector/jupyter_notebooks'

Make the parent of the current directory the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/cherry-mildew-detector'

## Set input directories

set paths to specific data folders required in notebook

In [5]:
raw_data_dir = 'inputs/datasets/raw/cherry-leaves'
train_data_dir = 'inputs/datasets/raw/cherry-leaves/train'
val_data_dir = 'inputs/datasets/raw/cherry-leaves/validation'
test_data_dir = 'inputs/datasets/raw/cherry-leaves/test'

print('the raw data directory is', raw_data_dir)
print('the training data directory is', train_data_dir)
print('the validation data directory is', val_data_dir)
print('the test data directory is', test_data_dir)


the raw data directory is inputs/datasets/raw/cherry-leaves
the training data directory is inputs/datasets/raw/cherry-leaves/train
the validation data directory is inputs/datasets/raw/cherry-leaves/validation
the test data directory is inputs/datasets/raw/cherry-leaves/test


## Set output directory

In [6]:
version = 'v1' # change version number for each iteration
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print('This version is already available - create a new version if required.')
    pass
else:
    os.makedirs(name=file_path)

This version is already available - create a new version if required.


## Set labels

In [7]:
labels = os.listdir(test_data_dir)
print('Labels for images are', labels)

Labels for images are ['healthy', 'powdery_mildew']


## Set image shape

In [8]:
image_128 = 'outputs/v1/image_shape_half.pkl'
image_50 = 'outputs/v1/image_shape_small.pkl'

---

# Data Augmentation

### Augment training image data and rescale validation and test image data



Define image size and batch size

In [9]:
# Allows size adjustment for tuning
IMG_SIZE = (128, 128) #image_128[:2] #can also be defined with any size eg.(128, 128)  
BATCH_SIZE = 32

Define data augmentation for training data

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=30,  # Random rotation up to 30 degrees
    width_shift_range=0.2,  # Shift width by 20%
    height_shift_range=0.2,  # Shift height by 20%
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Zoom into image
    horizontal_flip=True,  # Flip horizontally
    fill_mode='nearest'  # Fill empty space after transformations
)

Define rescaling for vallidation and test data

In [11]:
val_test_datagen = ImageDataGenerator(rescale=1./255)

Load datasets

In [12]:
train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  # One-hot encoding for softmax
)

val_set = val_test_datagen.flow_from_directory(
    val_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_set = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

train_set.class_indices

Found 2944 images belonging to 2 classes.
Found 420 images belonging to 2 classes.
Found 844 images belonging to 2 classes.


{'healthy': 0, 'powdery_mildew': 1}

---

# Hyperparameter Tuning

### Define base CNN model architecture and tune hyperparameters to optimise model

Define image shape

In [30]:
image_shape = (128, 128, 3)  # Modify as needed or use .pkl file

Define model for tuning

In [42]:
def build_model(hp):
    model = Sequential()

    # Input image shape
    model.add(Input(shape=image_shape))

    # First Conv2D layer with tunable filter size
    model.add(Conv2D(
        filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=32), 
        kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Conv2D layer
    model.add(Conv2D(
        filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=64), 
        kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    # Dense layer with tunable units
    model.add(Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))

    # Dropout layer with tunable rate
    model.add(Dropout(hp.Float('dropout', min_value=0.3, max_value=0.5, step=0.1)))

    # Output layer (2 neurons, Softmax activation)
    model.add(Dense(2, activation='softmax'))

    # Compile with tunable learning rate
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    return model

Initialise the tuner

In [43]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory= file_path,
    project_name='cherry_mildew_tuning'
)

Reloading Tuner from outputs/v1/cherry_mildew_tuning/tuner0.json


set early stop if not improving

In [44]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

Run tuner

In [45]:
tuner.search(train_set, validation_data=val_set, epochs=20, callbacks=[early_stop])

Export the best 10 hyperparameter results as a table

In [46]:
# Get the top 10 hyperparameter sets
best_hps = tuner.get_best_hyperparameters(num_trials=10)

# Extract hyperparameters into a DataFrame
hp_data = []
for i, hp in enumerate(best_hps):
    hp_data.append({
        'Trial': i + 1,
        'Conv1 Filters': hp.get('conv1_filters'),
        'Conv2 Filters': hp.get('conv2_filters'),
        'Dense Units': hp.get('dense_units'),
        'Dropout Rate': hp.get('dropout'),
        'Learning Rate': hp.get('learning_rate'),
    })

# Create DataFrame
df = pd.DataFrame(hp_data)

# Save as CSV in the output directory
csv_path = os.path.join(file_path, "hyperparameter_tuning_results.csv")
df.to_csv(csv_path, index=False)

# Display table
print(df)

print(f"Results saved to: {csv_path}")

   Trial  Conv1 Filters  Conv2 Filters  Dense Units  Dropout Rate  \
0      1             32            128          128           0.4   
1      2             64            256          128           0.5   
2      3            128            256          192           0.4   
3      4             96            192           64           0.3   
4      5             32            256          256           0.3   
5      6             96            256          256           0.3   
6      7             96            192           64           0.3   
7      8             32             64          128           0.4   
8      9             96            256          256           0.3   
9     10             96             64          128           0.3   

   Learning Rate  
0          0.001  
1          0.001  
2          0.001  
3          0.001  
4          0.001  
5          0.001  
6          0.001  
7          0.001  
8          0.001  
9          0.001  
Results saved to: outputs/v1/hy

Get the best model from hyperparameter tuning

In [47]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0] # best hyperparameters
best_model = tuner.hypermodel.build(best_hp) # build model from best hyperparameters

Display model summary

In [48]:
best_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,783,874 (56.40 MB)

 Trainable params: 14,783,874 (56.40 MB)

 Non-trainable params: 0 (0.00 B)

#### **NB: tuning results removed at this point as files too large and causing issues for workspace**

# Model training

### Uses best CNN model architecture from top hyperparameter tuning result

Define image shape

In [ ]:
image_shape = (128, 128, 3)  # Modify as needed or use .pkl file

Define best model

In [ ]:
def create_tf_model():
    model = Sequential()

    # Input image shape
    model.add(Input(shape=image_shape))

    # First Conv2D layer with 32 filters
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Conv2D layer with 128 filters
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Dense layer with 128 dense units
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))

    # Dropout layer with dropout rate of 0.4
    model.add(Dropout(0.4))

    # Output layer (2 neurons, Softmax activation)
    model.add(Dense(2, activation='softmax'))

    # Adam optimiser has default 0.001 learning rate
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


Set early stop if not improving

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

---

### Fit model for model training

In [ ]:
model = create_tf_model()
model.fit(train_set,
          epochs=25,
          validation_data=val_set,
          callbacks=[early_stop],
          verbose=1
          )

Save model

In [ ]:
model.save('outputs/v1/mildew_detector_model.h5')

---

# Conclusions and Next Steps

Conclusions: 
* 


Next steps:
* 